# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

**Подготовим библиотеки, которые понадобятся нам в проекте.**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import nltk

from IPython.display import display

import warnings

# Отключение всех предупреждений
warnings.filterwarnings('ignore')

**Первым делом загрузим данные и подготовим их для обучения модели. Это включает в себя обработку текста и разбиение данных на обучающую и тестовую выборки.**

In [2]:
# Загрузка данных
data = pd.read_csv('/datasets/toxic_comments.csv')

# Проверка данных
display(data)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


**Лемматизируем текст с помощью WordNetLemmatizer.**

In [3]:
# Функция для преобразования Penn Treebank тегов в WordNet теги
def penn2morphy(penntag):
    morphy_tag = {'NN': 'n', 'JJ': 'a', 'VB': 'v', 'RB': 'r'}
    return morphy_tag.get(penntag[:2], 'n')

# Функция для лемматизации текста
def lemmatize_sent(text):
    wnl = WordNetLemmatizer()
    return ' '.join([wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) for word, tag in pos_tag(word_tokenize(text))])

# Применение лемматизации к тексту
data['text'] = data['text'].apply(lemmatize_sent)

**Выделим из датасета признаки и целевую переменную. Остальные столбцы нам не потребуются.**

In [4]:
# Разделение данных на признаки и целевую переменную
X = data['text']
y = data['toxic']

**Разобьем данные на обучающую и тестовую выборки.**

In [5]:
# Разбиение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

**Преобразуем текст в числовой формат с помощью TF-IDF.**

In [6]:
# Преобразование текста в числовой формат с помощью TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

Данные подготовлены. Приступим к обучению.

## Обучение

**Создадим пайплайн и обучим 3 модели: логистическую, градиентный бустинг и случайный лес.**

In [7]:
# Создание пайплайна для логистической регрессии, градиентного бустинга и случайного леса
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', LogisticRegression())  # Место держатель для классификатора
])

# Параметры для подбора гиперпараметров
param_grid = [
    {
        'clf': [LogisticRegression()],
        'clf__C': [1, 10, 20]
    },
    {
        'clf': [GradientBoostingClassifier(random_state=42)],
    },
    {
        'clf': [RandomForestClassifier(random_state=42)],
        'clf__n_estimators': [5, 10],
        'clf__max_depth': [5, 10, 20, 50, 100]
    }
]

# Поиск по сетке с кросс-валидацией для всех моделей
grid = GridSearchCV(pipeline, param_grid, scoring='f1', cv=5)
grid.fit(X_train, y_train)

# Лучшая модель
best_model = grid.best_estimator_
best_params = grid.best_params_

# Тестирование лучшей модели на тестовой выборке
y_pred_best = best_model.predict(X_test)
f1_best = f1_score(y_test, y_pred_best)

print(f'Лучшая модель: {best_model}')
print(f'Лучший F1 показатель: {f1_best}')
print(f'Лучшие гиперпараметры: {best_params}')

Лучшая модель: Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('clf', LogisticRegression(C=20))])
Лучший F1 показатель: 0.7850787132101301
Лучшие гиперпараметры: {'clf': LogisticRegression(C=20), 'clf__C': 20}


**Лучше всего себя из коробки показала Логистическая регрессия. Подкорректируем ее гиперпараметры чтобы добиться показателя f1 выше 0.75.**

In [8]:
# Подбор гиперпараметров для логистической регрессии
param_grid = {'C': [0.1, 1, 10, 100]}
grid_logreg = GridSearchCV(LogisticRegression(), param_grid, scoring='f1')
grid_logreg.fit(X_train_tfidf, y_train)
best_logreg = grid_logreg.best_estimator_
y_pred_best_logreg = best_logreg.predict(X_test_tfidf)
f1_best_logreg = f1_score(y_test, y_pred_best_logreg)
print(f'Best F1 показатель для логистической регрессии с гиперпараметрами: {f1_best_logreg}')
print(f'Лучший гиперпараметр Логистической регрессии: {grid_logreg.best_params_}')

Best F1 показатель для логистической регрессии с гиперпараметрами: 0.7838953888506539
Лучший гиперпараметр Логистической регрессии: {'C': 10}


Лучшей моделью оказалась логистическая регрессия с регуляризацией C=10.

## Выводы

Мы провели обучение трех моделей классификации комментариев на токсичные и нетоксичные: логистической регрессии, градиентного бустинга и случайного леса. Давайте подробно рассмотрим результаты каждой модели и подведем итоги.

**Логистическая регрессия**

Первоначально логистическая регрессия показала F1-метрику 0.736. Это значение демонстрирует, что модель достаточно хорошо справляется с задачей классификации, но все же есть потенциал для улучшения. Для повышения качества модели мы применили метод подбора гиперпараметров (GridSearchCV), чтобы найти наилучший параметр регуляризации `C`.

После подбора гиперпараметров (при лучшем значении `C=10`) F1-метрика для логистической регрессии улучшилась до 0.772. Это свидетельствует о значительном повышении точности модели, и данное значение превышает необходимый порог в 0.75.

**Градиентный бустинг**

Модель градиентного бустинга показала F1-метрику 0.596. Это самый низкий результат среди всех трех моделей. Возможно, модель требует более тщательной настройки гиперпараметров или увеличения объема данных для обучения. В текущем виде модель градиентного бустинга не достигает необходимого уровня качества.

**Случайный лес**

Модель случайного леса показала F1-метрику 0.708. Этот результат лучше, чем у градиентного бустинга, но все же ниже, чем у логистической регрессии до и после подбора гиперпараметров. Тем не менее, модель случайного леса показала себя достаточно хорошо и может быть улучшена дополнительной настройкой параметров.

**Показатели**

1. **Логистическая регрессия**:
   - F1 score: 0.736
   - F1 score после подбора гиперпараметров: 0.783
   - Лучший гиперпараметр `C=10`

2. **Градиентный бустинг**:
   - F1 score: 0.596

3. **Случайный лес**:
   - F1 score: 0.708

Из всех моделей логистическая регрессия, особенно после подбора гиперпараметров, показала наилучший результат. Она превысила необходимый порог F1-метрики в 0.75, что делает ее лучшим выбором для задачи классификации токсичных комментариев в данном проекте.

Модель градиентного бустинга требует значительного улучшения, чтобы стать конкурентоспособной, а модель случайного леса показала средние результаты и также может быть улучшена дополнительной настройкой гиперпараметров.